In [102]:
!pip install pyspark

In [103]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

In [104]:
df = spark.read.csv(path = "../input/regresin-lineal/cruise_ship_info.csv", 
                    inferSchema = True, header = True)

df.show()

# Predecir la columna "crew"

In [105]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [106]:
indexer = StringIndexer(inputCol = "Cruise_line", outputCol = "Cruise_lineIndex")

df = indexer.fit(df).transform(df)
df.show(10)

In [107]:
# El nombre del barco no lo usaré.

assembler = VectorAssembler(inputCols = ["Age", "Tonnage", "passengers", "length", "cabins", "passenger_density", "Cruise_lineIndex"],
                            outputCol = "features")

output = assembler.transform(df)

output.select("features").show(truncate = False)

In [108]:
final_data = output.select("features", "crew")

train, test = final_data.randomSplit(weights = [0.8, 0.2], seed = 42)

In [109]:
train.describe().show()

test.describe().show()

In [110]:
lr = LinearRegression(featuresCol = "features",
                      labelCol = "crew", 
                      predictionCol = "predictions")


model = lr.fit(train)


print("Coeficientes: {}".format((model.coefficients)))
print("\n")
print("Intercepcion:{}".format((model.intercept)))

In [111]:
y_hat = model.evaluate(test)

In [112]:
y_hat.predictions.show()

In [113]:
y_hat.residuals.show()

In [114]:
print("RMSE: {}".format(y_hat.rootMeanSquaredError))
print("MSE: {}".format(y_hat.meanSquaredError))
print("r2: {}".format(y_hat.r2))